In [6]:
# importing the lib
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import numpy
import time
import sys
from selenium.webdriver.chrome.service import service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import re
import numpy as np
import warnings
warnings.filterwarnings("ignore")


In [7]:

def kayak(val,doj):
    url = val
    doj = doj
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito")
    driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=chrome_options)

    driver.maximize_window()
    driver.get(url)
    time.sleep(20)
    def click_more():

        for val in range(0,10):
            try:
                driver.find_element_by_xpath("//a[@class='moreButton']").click()
                time.sleep(4)
                val = val +1

            except:
                pass
    click_more()

    # depart-time base-time
    departure_time=[]
    departure_time= driver.find_elements_by_xpath("//ol[@class='flights']//span[@class='depart-time base-time']")

    arrival_time=[]
    arrival_time= driver.find_elements_by_xpath("//ol[@class='flights']//span[@class='arrival-time base-time']")

    duration =[]
    duration  = driver.find_elements_by_xpath("//ol[@class='flights']//div[@class='section duration allow-multi-modal-icons']//div[@class='top']")


    no_stop = []
    no_stop  = driver.find_elements_by_xpath("//ol[@class='flights']//div[@class='section stops']//div[@class='top']")

    price =[]
    price = driver.find_elements_by_xpath("//div[@class = 'booking']")

    airlines = []
    airlines = driver.find_elements_by_xpath("//span[@class = 'codeshares-airline-names']")

    airports = []
    airports = driver.find_elements_by_xpath("//ol[@class='flights']//span[@class='airport-name']")
    print("Airports len", len(airports))
    print("departure len", len(departure_time))



    # departure_time = driver.find_elements_by_xpath("//div[@class='itinerary-depart-time depart-time']")

    source_airport=[]
    destination_airport=[]


    price_list=[]
    stops=[]
    for i in range(len(departure_time)):
        departure_time[i]= departure_time[i].get_attribute('innerHTML')
        arrival_time[i] = arrival_time[i].get_attribute('innerHTML')
        duration[i] = duration[i].get_attribute('innerHTML')
        no_stop[i] = no_stop[i].get_attribute('innerHTML')
        l = re.findall("(direct|1|2|3|4|5)",no_stop[i])[0]
        stops.append(l)
        price[i] = price[i].get_attribute('innerHTML')
        b = re.findall("nbsp\;((\d\d\d)|(\d.\d\d\d)|(\d\d.\d\d\d))",price[i])[0][0]
        price_list.append(b)



    for j in range(len(airports)):
        if j % 2 ==0:
            a = airports[j].get_attribute("innerHTML")
            source_airport.append(re.findall("\\n...\\n",a)[0])
        else:
            a = airports[j].get_attribute("innerHTML")
            destination_airport.append(re.findall("\\n...\\n",a)[0])
    for k in range(len(airlines)):
        airlines[k] = airlines[k].get_attribute('innerHTML')

    print("source airport len", len(source_airport))


    doj1 = len(departure_time)*[doj]
    df  = pd.DataFrame(doj1,columns=["Date of journey"])
    df["Source"] = source_airport  
    df["Destination"] = destination_airport
    df["Departure_time"] = departure_time
    df["Arrival_time"] = arrival_time
    df["Duration"] = duration
    df["Stops"] = stops
    df["CAD $"]= price_list
    df["Airlines"] = airlines
    df["CAD $"]=df["CAD $"].astype(str).str.replace(",","")
    df["CAD $"] = pd.to_numeric(df['CAD $'])

    df["Source"]=df["Source"].astype(str).str.replace("\n","")
    df["Destination"]=df["Destination"].astype(str).str.replace("\n","")

    df["Duration"]=df["Duration"].astype(str).str.replace("\n","")
#     df = df.drop(df[(df.Stops =='3') | (df.Airlines == 'Multiple Airlines')].index)
    df = df.drop(df[(df.Airlines == 'Multiple Airlines')].index)
    df= df.drop_duplicates()
    df = df.sort_values(by=['Source','CAD $'])
    df = df.reset_index()
    df.index = df.index +1
    df.pop("index")
    driver.close()
    return (df)



In [8]:

# doj = "2022-05-30"
# date_list= ["2022-05-29","2022-05-30","2022-05-31","2022-05-25","2022-05-26","2022-05-27","2022-05-28"]
date_list= ["2022-05-29","2022-05-30","2022-05-31"]

for date in date_list:
    url_list = []
    doj  =date 
    l1 = "https://www.ca.kayak.com/flights/DEL,BOM-YYZ,YYC/"+doj+"?sort=bestflight_a"
    l2 = "https://www.ca.kayak.com/flights/DEL,BOM-YVR,YQB,YHZ/"+doj+"?sort=bestflight_a"
    l3 = "https://www.ca.kayak.com/flights/BLR,AMD,IXC-YYZ,YYC/"+doj+"?sort=bestflight_a"
    l4 = "https://www.ca.kayak.com/flights/IXC,BLR,AMD-YVR,YQB,YHZ/"+doj+"?sort=bestflight_a"
    url_list.append(l1)
    url_list.append(l2)
    url_list.append(l3)
    url_list.append(l4)
    
    for val in url_list:
        df = kayak(val,doj)
        df.to_csv('kayak4_data.csv', mode='a',index=False,header = False)





====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\Haet\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


Airports len 330
departure len 165
source airport len 165




====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\Haet\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


Airports len 332
departure len 166
source airport len 166




====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\Haet\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


...

Airports len 332
departure len 166
source airport len 166




====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\Haet\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


Airports len 332
departure len 166
source airport len 166




====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\Haet\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


Airports len 330
departure len 165
source airport len 165


In [41]:
df[df.Source == "DEL"]

,Date of journey,Source,Destination,Departure_time,Arrival_time,Duration,Stops,CAD $,Airlines
1,2022-05-30,DEL,YYZ,9:50 p.m.,3:40 p.m.,27h 20m,2,955,"Gulf Air, Air Transat"
2,2022-05-30,DEL,YYZ,2:35 a.m.,7:50 p.m.,26h 45m,2,985,Delta
3,2022-05-30,DEL,YYC,3:25 a.m.,12:03 a.m.,32h 08m,2,1007,"Qatar Airways, Alaska Airlines"
5,2022-05-30,DEL,YYZ,3:25 a.m.,8:31 p.m.,26h 36m,2,1067,"Qatar Airways, Air Canada"
6,2022-05-30,DEL,YYZ,3:25 a.m.,6:40 p.m.,24h 45m,2,1105,"Qatar Airways, Air Canada"
8,2022-05-30,DEL,YYZ,11:35 p.m.,1:09 p.m.,23h 04m,1,1149,United Airlines
9,2022-05-30,DEL,YYZ,11:35 p.m.,9:37 p.m.,31h 32m,1,1149,United Airlines
10,2022-05-30,DEL,YYZ,11:35 p.m.,3:20 p.m.,25h 15m,1,1149,United Airlines
11,2022-05-30,DEL,YYZ,11:35 p.m.,5:30 p.m.,27h 25m,1,1149,United Airlines
12,2022-05-30,DEL,YYZ,1:55 a.m.,10:22 p.m.,29h 57m,1,1149,United Airlines


In [28]:
a = source_airport[1]
re.findall("\\n...\\n",a)[0]

'\nBOM\n'

In [23]:
df

,Date of journey,Source,Destination,Departure_time,Arrival_time,Duration,Stops,CAD $,Airlines
1,2022-05-30,\nDEL\nIndira Gandhi Intl,\nYYZ\nPearson Intl,9:50 p.m.,3:40 p.m.,27h 20m,2,955,"Gulf Air, Air Transat"
2,2022-05-30,\nDEL\nIndira Gandhi Intl,\nYYZ\nPearson Intl,2:35 a.m.,7:50 p.m.,26h 45m,2,985,Delta
3,2022-05-30,\nDEL\nIndira Gandhi Intl,\nYYC\nCalgary,3:25 a.m.,12:03 a.m.,32h 08m,2,1007,"Qatar Airways, Alaska Airlines"
4,2022-05-30,\nBOM\nChhatrapati Shivaji Intl,\nYYZ\nPearson Intl,10:20 p.m.,3:40 p.m.,26h 50m,2,1058,"Emirates, Air Transat"
5,2022-05-30,\nDEL\nIndira Gandhi Intl,\nYYZ\nPearson Intl,9:50 p.m.,1:55 p.m.,25h 35m,2,1103,"Gulf Air, Air Canada"
...,...,...,...,...,...,...,...,...,...
77,2022-05-30,\nDEL\nIndira Gandhi Intl,\nYYZ\nPearson Intl,2:50 a.m.,11:45 a.m.,18h 25m,1,2496,Air Canada
78,2022-05-30,\nBOM\nChhatrapati Shivaji Intl,\nYYZ\nPearson Intl,4:35 a.m.,2:55 p.m.,19h 50m,1,2554,Etihad Airways
79,2022-05-30,\nDEL\nIndira Gandhi Intl,\nYYC\nCalgary,8:00 a.m.,3:43 p.m.,19h 13m,1,2723,Air Canada
80,2022-05-30,\nDEL\nIndira Gandhi Intl,\nYYZ\nPearson Intl,11:05 p.m.,6:40 a.m.,17h 05m,direct,2841,Air Canada


In [15]:
# destination = ["DEL","BOM","BLR","AMD","IXC"]
# source=  ["YYZ","YYC","YVR","YQB","YHZ"]
source = ["DEL"]
destination = ["YVR"]

doj=['2022-05-01']
df1 = pd.DataFrame(columns=['Date of journey', 'Source', 'Destination', 'Departure_time',
                            'Arrival_time', 'Duration', 'Stops', 'CAD $', 'Airlines'])
for d in range(len(doj)):
    for val in source:
        for values in destination:
            source_airport = val
            print(source_airport)


            destination_airport = values
            print(destination_airport)
    #         doj ='2022-05-30'
            df = kayak(doj[d],source_airport,destination_airport)
            print(df.shape)
    #         print("Departure length",len(departure_time))
    #         print(df1.describe())
    
            df.to_csv('kayak1_data.csv', mode='a',index=False,header = False)
    #         df1=pd.concat([df1,df])
    #         print(df1.shape())
        

DEL
YVR


TypeError: kayak() takes 2 positional arguments but 3 were given